In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Concatenate

In [2]:
# Create an inception module first
def inception_module(x,filter_1x1,filter_3x3_reduce,filter_3x3,
                     filter_5x5_reduce,filter_5x5,pool_proj):
    # 1x1 conv
    conv1 = Conv2D(filter_1x1,kernel_size=1,strides=1,activation='relu',padding='same')(x)

    # 1x1 conv => 3x3 conv
    conv3_a = Conv2D(filter_3x3_reduce,kernel_size=1,strides=1,activation='relu',padding='same')(x)
    conv3_b = Conv2D(filter_3x3,kernel_size=3,strides=1,activation='relu',padding='same')(conv3_a)

    # 1x1 conv => 5x5 conv
    conv5_a = Conv2D(filter_5x5_reduce,kernel_size=1,strides=1,activation='relu',padding='same')(x)
    conv5_b = Conv2D(filter_5x5_reduce,kernel_size=5,strides=1,activation='relu',padding='same')(conv5_a)

    # 3x3 max pooling => 1x1 conv
    pool = MaxPooling2D(pool_size=3,strides=1,padding='same')(x)
    pool_conv = Conv2D(pool_proj,kernel_size=1,strides=1,activation='relu',padding='same')(pool)

    # concatenate layers
    output = Concatenate(axis=-1)([conv1,conv3_b,conv5_b,pool_conv])
    return output

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout

In [4]:
# Create a GoogLeNet architecture
input_layer = Input(shape=(224,224,3))

x = Conv2D(filters=64,kernel_size=(7,7),strides=2,activation='relu',padding='same')(input_layer)
x = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(x)
x = Conv2D(filters=64,kernel_size=(1,1),strides=1,activation='relu',padding='valid')(x)
x = Conv2D(filters=192,kernel_size=(3,3),strides=1,activation='relu',padding='same')(x)
x = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(x)

# inception - 3x3 conv(3a)
x = inception_module(x,64,96,128,16,32,32)
# inception - 3x3 conv(3b)
x = inception_module(x,128,128,192,32,96,64)
# max pooling
x = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(x)

# inception - conv(4a)
x = inception_module(x,192,96,208,16,48,64)
# inception - conv(4b)
x = inception_module(x,160,112,224,24,64,64)
# inception - conv(4c)
x = inception_module(x,128,128,256,24,64,64)
# inception - conv(4d)
x = inception_module(x,112,144,288,32,64,64)
# inception - conv(4e)
x = inception_module(x,256,160,320,32,128,128)
# maxpooling
x = MaxPooling2D(pool_size=(3,3),strides=2,padding='same')(x)


# inception - conv(5a)
x = inception_module(x,256,160,320,32,128,128)
# inception - conv(5b)
x = inception_module(x,384,192,384,48,128,128)
# average pooling
x = AveragePooling2D(pool_size=(7,7),strides=1,padding='same')(x)
# dropout
x = Dropout(0.4)(x)
# linear
x = Flatten()(x)
# softmax
output = Dense(1000,activation='softmax')(x)

In [5]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
 99% 2.68G/2.70G [00:20<00:00, 265MB/s]
100% 2.70G/2.70G [00:20<00:00, 142MB/s]


In [6]:
!wget https://raw.githubusercontent.com/databyhuseyn/DeepLearning/refs/heads/main/helper_functions.py

--2025-04-26 16:03:47--  https://raw.githubusercontent.com/databyhuseyn/DeepLearning/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-04-26 16:03:47 (83.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [7]:
from helper_functions import unzip_data,walk_through_dir,plot_loss_curves

In [8]:
unzip_data('/content/new-plant-diseases-dataset.zip')

In [9]:
walk_through_dir('/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)')

There are 2 directories and 0 images in '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'.
There are 38 directories and 0 images in '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'.
There are 0 directories and 465 images in '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Corn_(maize)___healthy'.
There are 0 directories and 456 images in '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Cherry_(including_sour)___healthy'.
There are 0 directories and 478 images in '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Pepper,_bell___Bacterial_spot'.
There are 0 directories and 477 images in '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Corn_(maize)___Common_rust_'.
There are 0 directories and 432 images in '/content/New Plant Diseases Dataset

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
valid_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
IMG_SIZE = (224,224)
BATCH_SIZE = 32

In [11]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 batch_size=BATCH_SIZE,
                                                                 label_mode='categorical',
                                                                 crop_to_aspect_ratio=True,
                                                                 seed=42)

valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                 image_size=IMG_SIZE,
                                                                 batch_size=BATCH_SIZE,
                                                                 label_mode='categorical',
                                                                 crop_to_aspect_ratio=True,
                                                                 seed=42)

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


In [12]:
class_names = train_data.class_names
class_names

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [13]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

In [14]:
inputs = tf.keras.layers.Input(shape=(224,224,3))
aug_data = data_augmentation(inputs)
res = tf.keras.layers.Resizing(224,224)(aug_data)
preprocess = tf.keras.applications.efficientnet_v2.preprocess_input(res)
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet',
    input_tensor=preprocess
)

base_model.trainable=True
for layer in base_model.layers[:100]:
  layer.trainable=False

avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
dense = tf.keras.layers.Dense(128,activation='relu',kernel_initializer='he_normal')(avg)
dropout = tf.keras.layers.Dropout(0.5)(dense)
outputs = tf.keras.layers.Dense(len(class_names), 'softmax' if len(class_names) > 2 else 'sigmoid')(dropout)

model = tf.keras.Model(inputs,outputs)

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [15]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, None, None, 3)  │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resizing (Resizing)       │ (None, 224, 224, 3)    │              0 │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ resizing[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              0 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_project_conv      │ (None, 112, 112, 16)   │          4,608 │ stem_activation[0][0]  │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_project_bn        │ (None, 112, 112, 16)   │             64 │ block1a_project_conv[… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_project_activati… │ (None, 112, 112, 16)   │              0 │ block1a_project_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2a_expand_conv       │ (None, 56, 56, 64)     │          9,216 │ block1a_project_activ… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2a_expand_bn         │ (None, 56, 56, 64)     │            256 │ block2a_expand_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2a_expand_activation │ (None, 56, 56, 64)     │              0 │ block2a_expand_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2a_project_conv 

 Total params: 6,088,182 (23.22 MB)

 Trainable params: 5,344,006 (20.39 MB)

 Non-trainable params: 744,176 (2.84 MB)

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [17]:
model_ckp = tf.keras.callbacks.ModelCheckpoint(save_best_only=True,
                                               filepath='/content/plant_diseasses.keras')
model_es = tf.keras.callbacks.EarlyStopping(restore_best_weights=True,
                                            patience=5)

In [18]:
history = model.fit(train_data,
                    epochs=5,
                    validation_data=valid_data,
                    callbacks=[model_ckp,model_es])

Epoch 1/5
 667/2197 ━━━━━━━━━━━━━━━━━━━━ 1:05:24 3s/step - accuracy: 0.6898 - loss: 1.1770

KeyboardInterrupt: 